# The Kyle Model

In [4]:
import pickle
import numpy as np

The first thing to do is import the raw data - make sure you don't try to print any of this or jupyter *will* crash!

In [14]:
AAPLraw = pickle.load(open('AAPL.txt', 'rb'))
Craw = pickle.load(open('C.txt', 'rb'))
GBXraw = pickle.load(open('GBX.txt', 'rb'))
SPYraw = pickle.load(open('SPY.txt', 'rb'))
THOraw = pickle.load(open('THO.txt', 'rb'))

## Part 1

In [21]:
class Orderbook:
    '''
    The orderbook class. Initialized with a filename
    '''
    
    def __init__(self, filename):
        self.raw = pickle.load( open (filename))

In [9]:
dict1 = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
         'year': [2000, 2001, 2002, 2001, 2002],
         'pop': [1.5, 1.7, 3.6, 2.4, 2.9]}
dict1
pickle.dump(dict1, open('data/dict1.pkl', 'wb'))

In [10]:
HI = pickle.load(open('data/dict1.pkl', 'rb'))

In [ ]:
THOraw

In [11]:
print(SPYraw[0:2])

[(34260, {242.25: (10200, 'S', 34255.0261388, 7253328.0), 237.5: (302, 'B', 34202.9575079, 6292224.0), 2.0: (1, 'B', 28802.643090400004, 2535176.0), 244.61: (100, 'S', 34203.8539523, 6340504.0), 138.5: (800, 'B', 34203.017149, 6299972.0), 1.0: (2, 'B', 34202.2114565, 6207704.0), 228.82: (35, 'B', 34203.0132075, 6299204.0), 0.0: (100, 'B', 33900.7280436, 5144916.0), 234.56: (18, 'B', 34203.0208038, 6300668.0), 202.77: (160, 'B', 34203.0051707, 6297588.0), 224.77: (198, 'B', 34203.023326300005, 6301184.0), 224.5: (28, 'B', 34203.0137441, 6299340.0), 224.31: (10, 'B', 34203.0146368, 6299528.0), 232.51: (15, 'B', 34202.219410000005, 6208576.0), 210.15: (150, 'B', 34203.0247543, 6301432.0), 193.74: (1000, 'B', 34200.4424056, 5170116.0), 239.78: (7, 'B', 34203.0228139, 6301100.0), 241.72: (100, 'B', 23401.199621, 1383196.0), 210.81: (250, 'B', 34203.0160233, 6299792.0), 234.75: (30, 'B', 34202.2137875, 6207852.0), 243.88: (40, 'S', 34203.019047, 6300340.0), 243.64: (0, 'S', 34199.2986275, 58